In [1]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot

In [2]:
filepath = '../data/2018_Sales_Total.xlsx'
sales = pd.read_excel(filepath)

In [3]:
sales.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2018-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2018-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2018-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2018-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2018-01-01 23:26:55


In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   account number  1500 non-null   int64         
 1   name            1500 non-null   object        
 2   sku             1500 non-null   object        
 3   quantity        1500 non-null   int64         
 4   unit price      1500 non-null   float64       
 5   ext price       1500 non-null   float64       
 6   date            1500 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 82.2+ KB


In [5]:
sales.describe()

,account number,quantity,unit price,ext price
count,1500.000000,1500.000000,1500.000000,1500.000000
mean,485957.841333,24.308667,55.007527,1345.856213
std,223974.044572,14.439265,25.903267,1084.914881
min,141962.000000,-1.000000,10.030000,-97.160000
25%,257198.000000,12.000000,32.500000,472.177500
50%,527099.000000,25.000000,55.465000,1050.390000
75%,714466.000000,37.000000,77.075000,2068.330000
max,786968.000000,49.000000,99.850000,4824.540000


In [9]:
sales.columns = [re.sub(r'[\s\\,\?]+', '_', col.lower()) for col in sales.columns]
list(sales.columns)

['account_number',
 'name',
 'sku',
 'quantity',
 'unit_price',
 'ext_price',
 'date']

In [10]:
aggdf = sales.groupby('name')['ext_price'].agg(['mean', 'sum']).iloc[:5, :]
aggdf.head()

,mean,sum
name,,
Barton LLC,1334.615854,109438.50
"Cronin, Oberbrunner and Spencer",1339.321642,89734.55
"Frami, Hills and Schmidt",1438.466528,103569.59
"Fritsch, Russel and Anderson",1385.366790,112214.71
"Halvorson, Crona and Champlin",1206.971724,70004.36


## 1. Style numeric numbers

Refer this [cookbook](https://mkaz.blog/code/python-string-format-cookbook/) to find the formatting rules. ':' is the separator between the variable name and formatting string. See the example below:

In [40]:
a = 2.2222222
b = 3.3333333
f"{a:.2f} != {a:.3f} and {a:.3f} < {b:.3f}"

'2.22 != 2.222 and 2.222 < 3.333'

In [16]:
aggdf.style.format('{:.2f}')

,mean,sum
name,,
Barton LLC,1334.62,109438.50
"Cronin, Oberbrunner and Spencer",1339.32,89734.55
"Frami, Hills and Schmidt",1438.47,103569.59
"Fritsch, Russel and Anderson",1385.37,112214.71
"Halvorson, Crona and Champlin",1206.97,70004.36


In [17]:
aggdf.style.format('{:,.2f}')

,mean,sum
name,,
Barton LLC,"1,334.62","109,438.50"
"Cronin, Oberbrunner and Spencer","1,339.32","89,734.55"
"Frami, Hills and Schmidt","1,438.47","103,569.59"
"Fritsch, Russel and Anderson","1,385.37","112,214.71"
"Halvorson, Crona and Champlin","1,206.97","70,004.36"


In [18]:
# Use formatter dictionary to format multiple columns
formatter = {'mean': '{:.2f}', 'sum': '{:.3f}'}
aggdf.style.format(formatter)

,mean,sum
name,,
Barton LLC,1334.62,109438.500
"Cronin, Oberbrunner and Spencer",1339.32,89734.550
"Frami, Hills and Schmidt",1438.47,103569.590
"Fritsch, Russel and Anderson",1385.37,112214.710
"Halvorson, Crona and Champlin",1206.97,70004.360


In [19]:
# Doesn't display the index columns 
aggdf.iloc[:5, :].style.format('{:.2f}').hide_index()

mean,sum
1334.62,109438.50
1339.32,89734.55
1438.47,103569.59
1385.37,112214.71
1206.97,70004.36


If we want to look at total sales by each month, we can use the `resample` to summarize by month and also calculate how much each month is as a percentage of the total annual sales.

`resample` is `groupby` based on datetime. See [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html).

In [20]:
sales2 = sales.copy()
sales2.date = pd.to_datetime(sales2.date, format='%Y-%d-%m %H:%M:%s') # Add format is highly suggested, much faster
sales2.dtypes

account_number             int64
name                      object
sku                       object
quantity                   int64
unit_price               float64
ext_price                float64
date              datetime64[ns]
dtype: object

In [21]:
sales2.head()

,account_number,name,sku,quantity,unit_price,ext_price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2018-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2018-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2018-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2018-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2018-01-01 23:26:55


In [22]:
monthly_sales = sales2.resample('M', on='date')['ext_price'].agg('sum').reset_index()
# 'M' is the frequency, i.e. groupby month; on='date' can be ignored if we set 'date' as index
monthly_sales.head()

,date,ext_price
0,2018-01-31,185361.66
1,2018-02-28,146211.62
2,2018-03-31,203921.38
3,2018-04-30,174574.11
4,2018-05-31,165418.55


In [24]:
monthly_sales['pct_of_total'] = monthly_sales['ext_price'] / sales2['ext_price'].sum()
monthly_sales.head()

,date,ext_price,pct_of_total
0,2018-01-31,185361.66,0.091818
1,2018-02-28,146211.62,0.072426
2,2018-03-31,203921.38,0.101012
3,2018-04-30,174574.11,0.086475
4,2018-05-31,165418.55,0.081940


In [34]:
formatter = {'date': '{:%Y-%m}', 'ext_price': '{:,.0f}', 'pct_of_total': '{:.2%}'}
monthly_sales.style.format(formatter).hide_index()

date,ext_price,pct_of_total
2018-01,"185,362",9.18%
2018-02,"146,212",7.24%
2018-03,"203,921",10.10%
2018-04,"174,574",8.65%
2018-05,"165,419",8.19%
2018-06,"174,089",8.62%
2018-07,"191,662",9.49%
2018-08,"153,779",7.62%
2018-09,"168,443",8.34%
2018-10,"171,495",8.49%


## 2. Highlight the data cell

In addition to styling numbers, we can also style the cells in the DataFrame. 

### 2.1 Highlight max, min and null

You can highlight the max (`highlight_max`), min (`highlight_min`) and null (`highlight_null`) values of the dataframe. Refer to [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_max.html) for more details.

There are three control parameters:

- subset: a slice of columns/rows to highlight
- color: highlight color
- axis: axis of the dataframe

In [35]:
# Highlight the max and min using different colors
monthly_sales.style.format(formatter).hide_index()\
    .highlight_max(color='lightgreen')\
    .highlight_min(subset=['pct_of_total'], color='#cd4f39')  # Use subset to choose the columns to highlight

date,ext_price,pct_of_total
2018-01,"185,362",9.18%
2018-02,"146,212",7.24%
2018-03,"203,921",10.10%
2018-04,"174,574",8.65%
2018-05,"165,419",8.19%
2018-06,"174,089",8.62%
2018-07,"191,662",9.49%
2018-08,"153,779",7.62%
2018-09,"168,443",8.34%
2018-10,"171,495",8.49%


### 2.2 Highlight using gradient

Another useful function is the [`background_gradient`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.background_gradient.html) which can highlight the range of values in a column. Use `cmap` to choose a color palette for the gradient. The matplotlib [documentation](https://matplotlib.org/tutorials/colors/colormaps.html) lists all the available colormap options.

In [36]:
monthly_sales.style.format(formatter).hide_index()\
    .background_gradient(subset=['ext_price'], cmap='OrRd')

date,ext_price,pct_of_total
2018-01,"185,362",9.18%
2018-02,"146,212",7.24%
2018-03,"203,921",10.10%
2018-04,"174,574",8.65%
2018-05,"165,419",8.19%
2018-06,"174,089",8.62%
2018-07,"191,662",9.49%
2018-08,"153,779",7.62%
2018-09,"168,443",8.34%
2018-10,"171,495",8.49%


### 2.3 Highlight bar

The pandas styling function also supports drawing [bar](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.bar.html) charts within the columns. `bar` method has a few control parameters:

- color
- axis
- subset
- align: How to align the bars with the cells.

    * ‘left’ : the min value starts at the left of the cell.

    * ‘zero’ : a value of zero is located at the center of the cell.

    * ‘mid’ : the center of the cell is at (max-min)/2, or if values are all negative (positive) the zero is aligned at the right (left) of the cell
- vmin: Minimum bar value, defining the left hand limit of the bar drawing range, lower values are clipped to vmin. When None (default): the minimum value of the data will be used.
- vmax: Maximum bar value.

In [37]:
monthly_sales.style.format(formatter).hide_index()\
    .bar(color='#FFA07A', vmin=100_000, subset=['ext_price'], align='zero')\
    .bar(color='lightgreen', vmin=0, subset=['pct_of_total'], align='zero')\
    .set_caption('2018 Sales Performance')

date,ext_price,pct_of_total
2018-01,"185,362",9.18%
2018-02,"146,212",7.24%
2018-03,"203,921",10.10%
2018-04,"174,574",8.65%
2018-05,"165,419",8.19%
2018-06,"174,089",8.62%
2018-07,"191,662",9.49%
2018-08,"153,779",7.62%
2018-09,"168,443",8.34%
2018-10,"171,495",8.49%


## 3. Sparklines

`sparklines` is not a pandas built-in styling. Install this package and refer to this [example](https://pbpython.com/styling-pandas.html). It can shows mini distribution maps in the data cells and make the data easier to read.